# Importy

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Dane

W tym notatniku zamieszczono jedynie preprocessing danych, czyli wczytanie danych dostarczonych z excela, usunięcie zbędnych kolumn, dodanie ceny (targetu). Jest to początek notatnika data_analysis, który w większej części został zastąpiony przez pipeline w modelu.

Dane wyeksportowane poprzez ten notatnik są punktem wejścia do pipeline'u modelu. Przyszłe dane w inferencji należy dostarczać właśnie w takiej formie do modelu (oczywiście bez price). Mikroserwis powinien odpowiadać za przekształcenie danych z surowej postaci do takiej, jak wynikowa z tego notatnika, aby można je było następnie przekazać do modelu.

In [12]:
TRAIN_DATA_PATH = "../data/train_data.csv"
TEST_DATA_PATH = "../data/test_data.csv"

In [13]:
# podstawowe wczytanie plików z dostarczonego excela
listings = pd.read_csv('../data/raw/listings.csv', sep=';')
calendar = pd.read_csv('../data/raw/calendar.csv', sep=';')

# usunięcie $ i konwersja price na float
calendar['price'] = calendar['price'].str.replace(r'[\$,]', '', regex=True).astype(float)

In [14]:
# zostawiamy tylko wybrane kolumny
columns_to_keep = [
    'id',
    'neighbourhood_cleansed',
    'neighbourhood_group_cleansed',
    'latitude',
    'longitude',
    'property_type',
    'room_type',
    'accommodates',
    'bathrooms',
    'bathrooms_text',
    'bedrooms',
    'beds',
    'amenities',
    'instant_bookable',
    'price'
]

listings = listings[columns_to_keep]

In [15]:
print(listings.head())

            id           neighbourhood_cleansed neighbourhood_group_cleansed  \
0     16989407                la Vila de Gràcia                       Gràcia   
1     34133454                la Vila de Gràcia                       Gràcia   
2  1,04185E+18           la Dreta de l'Eixample                     Eixample   
3  6,95612E+17                      el Poblenou                   Sant Martí   
4       847195  l'Antiga Esquerra de l'Eixample                     Eixample   

    latitude  longitude                property_type        room_type  \
0  41.409920   2.157330         Private room in home     Private room   
1  41.397630   2.159340    Entire serviced apartment  Entire home/apt   
2  41.394798   2.165613           Entire rental unit  Entire home/apt   
3  41.399490   2.202610  Private room in rental unit     Private room   
4  41.385080   2.155270           Entire rental unit  Entire home/apt   

   accommodates  bathrooms    bathrooms_text  bedrooms  beds  \
0             2 

In [16]:
# chcemy dla każdego listingu mieć jedną cenę (price z listings_excel_ready nie spełnia tego, bo a)to jest 'local currency', a b)są braki dla niektórych listingów
# można usunąć tą kolumnę
listings.drop(columns=['price'], inplace=True)
# więc sprawdzamy czy każdy listing z listings_excel_ready ma przynajmniej jeden wpis w calendar
listings_ids = set(listings['id'])
calendar_listing_ids = set(calendar['listing_id'])

missing_in_calendar = listings_ids - calendar_listing_ids

if missing_in_calendar:
    print(f"Brak wpisów w calendar dla listingów o ID: {missing_in_calendar}")
else:
    print("Każdy listing z listings_excel_ready ma wpis w calendar.")

Każdy listing z listings_excel_ready ma wpis w calendar.


In [17]:
# skoro tak to możemy tego użyć - chcemy połączyć listingi z odpowiednią ceną, sprawdzamy ile jest unikalnych wartości
price_stats = calendar.groupby('listing_id')['price'].agg(
    nunique='nunique',
    min='min',
    max='max',
    mean='mean',
    median='median'
).reset_index()

# niektóre listingi mają różne ceny w zależnosci od dnia - my chcemy jedną wartość, dlatego dla tych przypadków bierzemy medianę
price_stats['final_price'] = price_stats.apply(
    lambda row: row['median'] if row['nunique'] > 1 else row['min'],
    axis=1
)

In [18]:
# dodajemy kolumnę
listings = listings.merge(price_stats[['listing_id', 'final_price']], left_on='id', right_on='listing_id', how='left')

listings.drop(columns=['listing_id'], inplace=True)
listings.rename(columns={'final_price': 'price'}, inplace=True)

In [19]:
# Na tym etapie można zapisać dane
train_df, test_df = train_test_split(listings, test_size=0.1, random_state=42)

train_df.to_csv(TRAIN_DATA_PATH, index=False, sep=';')
test_df.to_csv(TEST_DATA_PATH, index=False, sep=';')

In [20]:
print(train_df['price'].median())

100.0
